In [1]:

import pandas as pd
import numpy as np
import bs4 
import csv
import requests
from urllib.request import urlopen as uReq
from bs4 import BeautifulSoup as soup


In [2]:
my_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'

In [3]:
uClient = uReq(my_url) #opening connection grabbing page

In [4]:
page_html = uClient.read()
uClient.close()
page_soup = soup(page_html,'html.parser') #html parsing


In [5]:
page_soup.h1 #testing

<h1 class="firstHeading" id="firstHeading" lang="en">List of postal codes of Canada: M</h1>

In [6]:
page_soup.td

<td>M1A</td>

In [7]:
containers = page_soup.findAll("table",{"class": "wikitable sortable jquery-tablesorter"})
len(containers)

0

In [8]:
containers_2 = page_soup.findAll("td")
len(containers_2)
    
    

898

In [9]:
table = page_soup.find("table")

In [10]:
output_rows = []
for table_row in table.findAll('tr'):
    columns = table_row.findAll('td')
    output_row = []
    for column in columns:
        output_row.append(column.text)
    output_rows.append(output_row)
    


In [11]:

with open('output.csv', 'w') as csvfile:
    writer = csv.writer(csvfile)
    writer.writerows(output_rows)

In [12]:
df = pd.read_csv('output.csv', names = ['Postcode','Borough','Neighborhood'])
df

,Postcode,Borough,Neighborhood
0,M1A,Not assigned,Not assigned\n
1,M2A,Not assigned,Not assigned\n
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n
7,M6A,North York,Lawrence Manor\n
8,M7A,Queen's Park,Not assigned\n
9,M8A,Not assigned,Not assigned\n


In [13]:
notAssigned = df[ df['Borough'] == 'Not assigned' ].index
df.drop(notAssigned, inplace=True)
df.head()
#remove not assigned Bouroughs

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods\n
3,M4A,North York,Victoria Village\n
4,M5A,Downtown Toronto,Harbourfront\n
5,M5A,Downtown Toronto,Regent Park\n
6,M6A,North York,Lawrence Heights\n


In [14]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 211 entries, 2 to 286
Data columns (total 3 columns):
Postcode        211 non-null object
Borough         211 non-null object
Neighborhood    211 non-null object
dtypes: object(3)
memory usage: 6.6+ KB


In [15]:
df.Neighborhood = df.Neighborhood.str.replace('\n','')
df.head()
#remove /n                                            

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront
5,M5A,Downtown Toronto,Regent Park
6,M6A,North York,Lawrence Heights


In [16]:
df.Neighborhood = df.Neighborhood.str.replace('Not assigned','Queens Park')
df['Neighborhood'] = df[['Postcode','Borough','Neighborhood']].groupby(['Postcode','Borough'])['Neighborhood'].transform(lambda x: ','.join(x))
clean_df = df[['Postcode','Borough','Neighborhood']].drop_duplicates()
clean_df
#group and concatenate Neighborhood fields, rename new dataframe

,Postcode,Borough,Neighborhood
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Harbourfront,Regent Park"
6,M6A,North York,"Lawrence Heights,Lawrence Manor"
8,M7A,Queen's Park,Queens Park
10,M9A,Etobicoke,Islington Avenue
11,M1B,Scarborough,"Rouge,Malvern"
14,M3B,North York,Don Mills North
15,M4B,East York,"Woodbine Gardens,Parkview Hill"
17,M5B,Downtown Toronto,"Ryerson,Garden District"


In [17]:
clean_df.shape
#newdataframe shape

(103, 3)